# Extracción de datos de base Activity Info 2022

In [33]:
#!/usr/bin/env python
# coding: utf-8
import activityinfo as AT
import pandas as pd
import numpy as np
import geopandas as gpd
import time
import pickle
import re

In [34]:
t = time.localtime()
timestamp = time.strftime('%b-%d-%Y_%H%M', t)

# Leer tablas de referencia

In [35]:
Admin_ref = pd.read_excel(r'C:\Users\jorcabrera\OneDrive - International Organization for Migration - IOM\InteragencyGTRM\GTRM 2022\IM 2022\ActivityInfo\MasterList\NEW_MasterList2022.xlsx', sheet_name='Admin')
Organizations_ref = pd.read_excel(r'C:\Users\jorcabrera\OneDrive - International Organization for Migration - IOM\InteragencyGTRM\GTRM 2022\IM 2022\ActivityInfo\MasterList\NEW_MasterList2022.xlsx', sheet_name='ImplementingPartners')
MarcoMonitoreo_ref= pd.read_excel(r'C:\Users\jorcabrera\OneDrive - International Organization for Migration - IOM\InteragencyGTRM\GTRM 2022\IM 2022\ActivityInfo\MasterList\NEW_MasterList2022.xlsx', sheet_name='marco_monitoreo')

indicadores_ref=MarcoMonitoreo_ref[['codigo', 'codigo_regional','indicador_eng', 'unidad','unidad_reg', 'capacitacion']]

## Extraccion de datos

In [36]:
t = time.localtime()
timestamp = time.strftime('%b-%d-%Y_%H%M', t)
mes_curso = 'AI2022-03.xlsx'
mes = '2022-03'
df_activity_info = pd.read_excel(r'C:\Users\jorcabrera\OneDrive - International Organization for Migration - IOM\InteragencyGTRM\GTRM 2022\IM 2022\ActivityInfo\DescargasMensuales'+ '\\' + mes_curso)
df_activity_info = df_activity_info[df_activity_info['Value'] != 0]


In [37]:
#Filtro MES
#df_activity_info = df_activity_info[(df_activity_info['mes'] == mes)]

# Transferencias monetarias
Al hacer este paso se duplica el monto por cada indicador, esto debe ser considerado luego al realizar los tableros

In [38]:
df_CVA = df_activity_info[df_activity_info['Quantity Field Code'] == 'usd_transfer']
df_CVA = df_CVA[['RecordId', 'mecanismo_de_entrega', 'Quantity Field Code', 'Value']]
df_CVA.rename(columns={'Quantity Field Code': 'CVA_tag', 'Value': 'Monto transferido'}, inplace=True)
del df_CVA['mecanismo_de_entrega']
df_activity_info = df_activity_info[df_activity_info['Quantity Field Code'] != 'usd_transfer']
df_activity_info = pd.merge(df_activity_info, df_CVA, how='left', on='RecordId')
df_activity_info['CVA_tag'] = np.where(((df_activity_info['modalidad_de_implementación'] == 'Efectivo')
|
(df_activity_info['modalidad_de_implementación'] == 'Cupón / voucher')
),'Si','No')


Tagging de indicadores segun codigo

Creación de variables para nombres de las columnas

In [39]:
poblacion_meta_col = 'población meta'
unidad = 'unidad'
frecuencia = 'frecuencia'
monitoreo_frontera ='monitoreo_frontera'
lgbti = 'lgbti'
discapacitados = 'discapacidad'

## Definición de poblacion meta
Se clasifica segun el código de indicador el tipo de población que se va a medir con el indicador

In [40]:
df_activity_info.loc[(df_activity_info['Quantity Field Code'].str.contains("RM", regex=False) == True),
                         poblacion_meta_col] = 'Refugiados y migrantes'
df_activity_info.loc[(df_activity_info['Quantity Field Code'].str.contains("CA", regex=False) == True),
                         poblacion_meta_col] = 'Comunidad de acogida'

## Definición de unidades
Se clasifica segun el código de indicador la unidad en la que se expresa el indicador ej. Niñas Niños Mujeres Hombres 

In [41]:
df_activity_info.loc[(df_activity_info['Quantity Field Code'].str.contains("NA", regex=False) == True),
                         unidad] = 'Niñas'
df_activity_info.loc[(df_activity_info['Quantity Field Code'].str.contains("ADM", regex=False) == True),
                         unidad] = 'Niñas'
df_activity_info.loc[(df_activity_info['Quantity Field Code'].str.contains("NN", regex=False) == True),
                         unidad] = 'Niños'
df_activity_info.loc[(df_activity_info['Quantity Field Code'].str.contains("ADH", regex=False) == True),
                         unidad] = 'Niños'
df_activity_info.loc[(df_activity_info['Quantity Field Code'].str.contains("MM", regex=False) == True),
                         unidad] = 'Mujeres'
df_activity_info.loc[(df_activity_info['Quantity Field Code'].str.contains("HH", regex=False) == True),
                         unidad] = 'Hombres'
df_activity_info.loc[(df_activity_info['Quantity Field Code'].str.contains("ADOTR", regex=False) == True),
                         unidad] = 'Otros'
df_activity_info.loc[(df_activity_info['Quantity Field Code'].str.contains("OTR", regex=False) == True),
                         unidad] = 'Otros'

Se agregan campos de unidades extra, para indicadores diferentes de poblacion meta

In [42]:
df_activity_info = pd.merge(df_activity_info, indicadores_ref, how='left', left_on='codigo',
                                    right_on='codigo')
df_activity_info.rename(columns={'unidad_x': 'unidad', 'unidad_y': 'unidad_aux'}, inplace=True)

Asignacion de nuevos y recurrentes

In [43]:
df_activity_info.loc[(df_activity_info['Quantity Field Code'].str.endswith('_U') == True),
                         frecuencia] = 'Nuevo'
df_activity_info.loc[(df_activity_info['Quantity Field Code'].str.endswith('_N') == True),
                         frecuencia] = 'Nuevo'
df_activity_info.loc[(df_activity_info['Quantity Field Code'].str.endswith('_PR') == True),
                         frecuencia] = 'Nuevo'
df_activity_info.loc[(df_activity_info['Quantity Field Code'].str.endswith('_R') == True),
                         frecuencia] = 'Recurrente'
df_activity_info.loc[(df_activity_info[frecuencia].isnull() == True), frecuencia] = 'Nuevo'

# Organizaciones

Implementacion directa: Se hace el llenado acutomatico usando la informacion de las columas de organización y de implementador

In [44]:
df_activity_info['implementador.nombre'] = np.where(df_activity_info['implementación'] == 'Directa',
                                                 df_activity_info['organizacion.nombre'],
                                                 df_activity_info['implementador.nombre'])

In [45]:

Organizations_ref = Organizations_ref[Organizations_ref['Nombre_ecu'].notna()]
Organizations_ref = Organizations_ref[['Nombre_ecu','Name','Acronym/Short Name']]
## DONANTE
df_activity_info = pd.merge(df_activity_info, Organizations_ref, how='left', left_on='donante.nombre',
                                    right_on='Nombre_ecu')
df_activity_info.rename(columns={'Acronym/Short Name': 'Acronimo Donante'}, inplace=True)
df_activity_info.drop(['Nombre_ecu','Name'], axis=1, inplace=True)
## IMPLEMENTADOR
df_activity_info = pd.merge(df_activity_info, Organizations_ref, how='left', left_on='implementador.nombre',
                                    right_on='Nombre_ecu')
df_activity_info.rename(columns={'Acronym/Short Name': 'Acronimo Implementador'}, inplace=True)
df_activity_info.drop(['Nombre_ecu','Name'], axis=1, inplace=True)
## IMPLEMENTADOR EXTRA
df_activity_info['implementador_extra.nombre'] =df_activity_info['implementador_extra.nombre'].astype(str)
df_activity_info = pd.merge(df_activity_info, Organizations_ref, how='left', left_on='implementador_extra.nombre',
                                    right_on='Nombre_ecu')
df_activity_info.rename(columns={'Acronym/Short Name': 'Acronimo Implementador Extra'}, inplace=True)
df_activity_info.drop(['Nombre_ecu','Name'], axis=1, inplace=True)

# GTRM

In [46]:
## Archivos Geograficos
ruta_dpa_canton_ecuador = "shapefiles/dpa_canton_codificado_v1_4326.shp"
dpa_ecuador_cantones = gpd.read_file(ruta_dpa_canton_ecuador, encoding='utf-8')
df_activity_info['DPA_CANTON'] = df_activity_info['canton.code'].str[2:6]
df_activity_info = pd.merge(dpa_ecuador_cantones, df_activity_info, how='left', on='DPA_CANTON')
df_activity_info.drop_duplicates(keep='first', inplace=True)
del df_activity_info['geometry']

In [47]:
df_activity_info['unidad_combinada'] = np.where(((df_activity_info['unidad'] == 'Niñas')
|
(df_activity_info['unidad'] == 'Niños')
|
(df_activity_info['unidad'] == 'Mujeres')
|
(df_activity_info['unidad'] == 'Hombres')
|
(df_activity_info['unidad'] == 'Otros')
),df_activity_info['unidad'],df_activity_info['unidad_aux'])

In [48]:
df_activity_info.to_excel(r'C:\Users\jorcabrera\OneDrive - International Organization for Migration - IOM\InteragencyGTRM\GTRM 2022\IM 2022\ActivityInfo\DescargasMensuales\procesados\proc_'+mes_curso)